In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import linear_model
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import datetime as dt

In [2]:
florida = pd.read_csv("Florida_all_data.csv", index_col = 0)

In [3]:
florida.head()

,SCAPN,SCSitus_NumNam_x,PropID,Star_rating,City,State,Zip,County,For_sale_price,For_sale_status,...,Total_value,Assessed_yr,Floor_area_ratio,Num_floors,Price_per_sqft,Land_price_per_AC,Sale_date,Sale_price,Listing_broker_company,10_yr_treasury_rate
0,FL Pinellas 34-29-15-50256-001-0010,FL Pinellas600 Bay Dr,P2828,4,Largo,FL,337703328,Pinellas,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FL Pinellas 24-31-16-29718-013-0150,FL Pinellas1601 Central Ave,P2829,4,Saint Petersburg,FL,337138934,Pinellas,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,FL Hillsborough A-24-29-18-4ZI-000017-00002.1,FL Hillsborough915 Franklin St,P2830,4,Tampa,FL,336023809,Hillsborough,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FL Hillsborough A-19-29-19-4X2-000010-00001.0,FL Hillsborough1211 Kennedy Blvd,P2831,4,Tampa,FL,336023513,Hillsborough,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FL Pinellas 19-31-17-74466-021-0141,FL Pinellas297 1st Ave,P2832,4,Saint Petersburg,FL,337013308,Pinellas,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
florida.columns.unique()

Index(['SCAPN', 'SCSitus_NumNam_x', 'PropID', 'Star_rating', 'City', 'State',
       'Zip', 'County', 'For_sale_price', 'For_sale_status', 'Land_area',
       'Num_units', 'Cap_rate', 'Vacancy_%', 'Avg_unit_sqft',
       'Avg_Concession_%', 'Parking_per_unit', 'Days_on_mkt', 'Amenities',
       'Year_built', 'Year_reno', 'Last_sale_date', 'Last_sale_price',
       'Transit_dist_mile', 'Transit_walk_time_min', 'Const_material',
       'Num_studio', 'Studio_rent_sqft', 'Studio_avg_sqft',
       'Studio_concession_%', 'Studio_vacancy_%', 'Num_1_bed',
       '1_bed_rent_sqft', '1_bed_avg_sqft', '1_bed_concession_%',
       '1_bed_vacancy_%', 'Num_2_bed', '2_bed_rent_sqft', '2_bed_avg_sqft',
       '2_bed_concession_%', '2_bed_vacancy_%', 'Num_3_bed', '3_bed_rent_sqft',
       '3_bed_avg_sqft', '3_bed_concession_%', '3_bed_vacancy_%', 'Num_4_bed',
       '4_bed_rent_sqft', '4_bed_avg_sqft', '4_bed_concession_%',
       '4_bed_vacancy_%', '2010_Avg_age', '2019_Avg_age', '2019_Pop_25_39',
   

In [48]:
for i in florida["10_yr_treasury_rate"]: 
    float(i)

ValueError: could not convert string to float: '.'

In [54]:
fl_reg_data = florida[["Star_rating", 
                       "City",
                       "County",
                       "Land_area",
                       "Vacancy_%",
                       "Num_studio",
                       "Avg_unit_sqft",
                       "Parking_per_unit",
                      "Year_built",
                      "Transit_dist_mile",
                      "Price_per_sqft",
                      "10_yr_treasury_rate"]]

In [55]:
fl = fl_reg_data.dropna()

In [56]:
fl.shape

(268, 12)

In [8]:
fl.head()

,Star_rating,City,County,Land_area,Vacancy_%,Num_studio,Avg_unit_sqft,Parking_per_unit,Year_built,Transit_dist_mile,Price_per_sqft,10_yr_treasury_rate
11,4,Saint Petersburg,Pinellas,4.3464,5.58,36.0,883.0,0.844,2014.0,13.98,152.81,2.30
12,4,Saint Petersburg,Pinellas,4.3464,5.58,36.0,883.0,0.844,2014.0,13.98,56.01,2.30
13,3,Saint Petersburg,Pinellas,0.2748,2.77,83.0,548.0,0.266,1971.0,13.74,52.06,3.53
16,4,Saint Petersburg,Pinellas,2.8921,5.89,16.0,989.0,0.908,2016.0,13.23,354.79,2.54
30,4,Saint Petersburg,Pinellas,3.8851,18.94,283.0,459.0,0.624,1971.0,12.70,203.53,2.48


In [24]:
type(fl.iloc[0,-1])

str

In [26]:
fl["10_yr_treasury_rate"] = fl["10_yr_treasury_rate"].astype(float)

/Users/jennyshang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [57]:
Y = fl["Price_per_sqft"]
X = fl.drop(["Price_per_sqft","City","County","Star_rating","Vacancy_%","Avg_unit_sqft","Land_area", "10_yr_treasury_rate"], axis = 1)

X = sm.add_constant(X)
model = sm.OLS(Y.astype(float), X.astype(float)).fit() 
predictions = model.predict(X)

In [58]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         Price_per_sqft   R-squared:                       0.349
Model:                            OLS   Adj. R-squared:                  0.339
Method:                 Least Squares   F-statistic:                     35.17
Date:                Fri, 06 Dec 2019   Prob (F-statistic):           1.58e-23
Time:                        15:16:14   Log-Likelihood:                -1450.3
No. Observations:                 268   AIC:                             2911.
Df Residuals:                     263   BIC:                             2928.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const             -3606.7968    419.452     -8.599      0.000   -4432.708   -2780.885
Num_studio            0.1618      0.088      1.837      0.067      -0.012       0.335
Parking_per_unit    -27.0970      5.405     -5.013      0.000     -37.739     -16.455
Year_built            1.8916      0.211      8.959      0.000       1.476       2.307
Transit_dist_mile    -1.4063      0.329     -4.280      0.000      -2.053      -0.759
==============================================================================
Omnibus:                       57.826   Durbin-Watson:                   1.767
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              139.103
Skew:                           1.011   Prob(JB):                     6.23e-31
Kurtosis:                       5.893   Cond. No.                     2.49e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.49e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [59]:
fl_reg_data = florida[["Star_rating", 
                       "City",
                       "County",
                       "Land_area",
                       "Vacancy_%",
                       "Num_1_bed",
                       "Avg_unit_sqft",
                       "Parking_per_unit",
                      "Year_built",
                      "Transit_dist_mile",
                      "Price_per_sqft",
                      "10_yr_treasury_rate"]]

In [60]:
fl = fl_reg_data.dropna()

In [61]:
fl.shape

(1929, 12)

In [62]:
Y = fl["Price_per_sqft"]
X = fl.drop(["Price_per_sqft","City","County","Star_rating","Vacancy_%","Avg_unit_sqft","Land_area", "10_yr_treasury_rate"], axis = 1)

X = sm.add_constant(X)
model = sm.OLS(Y.astype(float), X.astype(float)).fit() 
predictions = model.predict(X)

In [63]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         Price_per_sqft   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     11.49
Date:                Fri, 06 Dec 2019   Prob (F-statistic):           3.23e-09
Time:                        15:16:44   Log-Likelihood:                -14460.
No. Observations:                1929   AIC:                         2.893e+04
Df Residuals:                    1924   BIC:                         2.896e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const             -2480.5196   1356.121     -1.829      0.068   -5140.142     179.102
Num_1_bed             0.7262      0.135      5.371      0.000       0.461       0.991
Parking_per_unit    -36.1865     15.575     -2.323      0.020     -66.733      -5.640
Year_built            1.2994      0.683      1.902      0.057      -0.040       2.639
Transit_dist_mile    -1.8390      0.994     -1.849      0.065      -3.789       0.111
==============================================================================
Omnibus:                     5764.195   Durbin-Watson:                   1.988
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        256944152.271
Skew:                          41.532   Prob(JB):                         0.00
Kurtosis:                    1789.034   Cond. No.                     2.72e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.72e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [64]:
fl_reg_data = florida[["Star_rating", 
                       "City",
                       "County",
                       "Land_area",
                       "Vacancy_%",
                       "Num_2_bed",
                       "Avg_unit_sqft",
                       "Parking_per_unit",
                      "Year_built",
                      "Transit_dist_mile",
                      "Price_per_sqft",
                      "10_yr_treasury_rate"]]

In [65]:
fl = fl_reg_data.dropna()
fl.shape

(2035, 12)

In [66]:
Y = fl["Price_per_sqft"]
X = fl.drop(["Price_per_sqft","City","County","Star_rating","Vacancy_%","Avg_unit_sqft","Land_area", "10_yr_treasury_rate"], axis = 1)

X = sm.add_constant(X)
model = sm.OLS(Y.astype(float), X.astype(float)).fit() 
predictions = model.predict(X)

In [67]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         Price_per_sqft   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     10.36
Date:                Fri, 06 Dec 2019   Prob (F-statistic):           2.62e-08
Time:                        15:17:21   Log-Likelihood:                -15205.
No. Observations:                2035   AIC:                         3.042e+04
Df Residuals:                    2030   BIC:                         3.045e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const             -2102.7534   1302.599     -1.614      0.107   -4657.324     451.817
Num_2_bed             0.6174      0.126      4.901      0.000       0.370       0.864
Parking_per_unit    -40.1051     14.564     -2.754      0.006     -68.668     -11.543
Year_built            1.1128      0.657      1.695      0.090      -0.175       2.401
Transit_dist_mile    -1.8463      0.942     -1.960      0.050      -3.693       0.001
==============================================================================
Omnibus:                     6135.666   Durbin-Watson:                   1.991
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        301815214.061
Skew:                          42.662   Prob(JB):                         0.00
Kurtosis:                    1887.731   Cond. No.                     2.75e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.75e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""